# Invasive Birds Project
## Notebook 1: Data Prep
Original data source: [Project FeederWatch Raw Dataset Downloads](https://feederwatch.org/explore/raw-dataset-requests/) 

[![feederwatch banneer image](images/feederwatch_banner.jpg)](https://feederwatch.org/)  
Click the banner for more information on Project FeederWatch.

---
### About the data:
The FeederWatch data consist of seven rather sizeable datasets, spanning from 1987 to 2024 (at the time of this writing). They provide a very wide scope of information, not all of which is needed for this particular project. In addition, files of this magnitude far exceed the limits of GitHub. All that in consideration, the end goal of this notebook is to create a dataframe that meets these criteria.

### Getting started: the preliminaries

In [2]:
import pandas as pd
import numpy as np
import glob

pd.options.mode.chained_assignment = None # disable chain assignment warnings

## Initial inspection of a dataset

Information provided by Project FeederWatch, included in **FeederWatch_Data_Dictionary.xlsx**, indicates that the structure of all of the checklist tables are the same. Before further work with all of the files, a preview of a single file will help determine which data will be useful for the project and what can be further derived from it.

In [2]:
#load first dataset for inspection

preview = pd.read_csv('bird_datasets/Project FeederWatch/all_years/PFW_all_1988_1995_June2023_Public.csv', low_memory = False)
pd.set_option('display.max_columns', None)

preview.head()

,LOC_ID,LATITUDE,LONGITUDE,SUBNATIONAL1_CODE,ENTRY_TECHNIQUE,SUB_ID,OBS_ID,Month,Day,Year,PROJ_PERIOD_ID,SPECIES_CODE,alt_full_spp_code,HOW_MANY,PLUS_CODE,VALID,REVIEWED,DAY1_AM,DAY1_PM,DAY2_AM,DAY2_PM,EFFORT_HRS_ATLEAST,SNOW_DEP_ATLEAST,Data_Entry_Method
0,L103924,42.557469,-83.199715,US-MI,PointMaker1.0_2,S119768,OBS6669852,1,11,1992,PFW_1992,houfin,NaN,4.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,NaN
1,L103924,42.557469,-83.199715,US-MI,PointMaker1.0_2,S119768,OBS6669853,1,11,1992,PFW_1992,houspa,NaN,26.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,NaN
2,L103924,42.557469,-83.199715,US-MI,PointMaker1.0_2,S119768,OBS7763087,1,11,1992,PFW_1992,moudov,NaN,5.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,NaN
3,L103924,42.557469,-83.199715,US-MI,PointMaker1.0_2,S119768,OBS5875613,1,11,1992,PFW_1992,eursta,NaN,35.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,NaN
4,L103924,42.557469,-83.199715,US-MI,PointMaker1.0_2,S119768,OBS5875614,1,11,1992,PFW_1992,daejun,NaN,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,NaN


In [3]:
preview.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2950137 entries, 0 to 2950136
Data columns (total 24 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   LOC_ID              2950137 non-null  object 
 1   LATITUDE            2950137 non-null  float64
 2   LONGITUDE           2950137 non-null  float64
 3   SUBNATIONAL1_CODE   2950137 non-null  object 
 4   ENTRY_TECHNIQUE     2950137 non-null  object 
 5   SUB_ID              2950137 non-null  object 
 6   OBS_ID              2947696 non-null  object 
 7   Month               2950137 non-null  int64  
 8   Day                 2950137 non-null  int64  
 9   Year                2950137 non-null  int64  
 10  PROJ_PERIOD_ID      2950137 non-null  object 
 11  SPECIES_CODE        2950137 non-null  object 
 12  alt_full_spp_code   22519 non-null    object 
 13  HOW_MANY            2947696 non-null  float64
 14  PLUS_CODE           2947075 non-null  float64
 15  VALID          

#### Space and time

In [4]:
preview.SUBNATIONAL1_CODE.unique()

array(['US-MI', 'CA-ON', 'US-CO', 'US-IL', 'US-MD', 'US-CA', 'US-ME',
       'US-SC', 'US-OH', 'US-VA', 'US-AL', 'US-MO', 'US-NE', 'US-CT',
       'US-MA', 'US-NY', 'US-WV', 'US-TN', 'US-SD', 'US-MN', 'US-TX',
       'US-WA', 'US-ID', 'US-FL', 'US-OK', 'US-IN', 'US-WI', 'US-AZ',
       'US-NC', 'US-PA', 'US-IA', 'US-DE', 'US-GA', 'US-NM', 'US-KS',
       'US-NJ', 'US-KY', 'US-NH', 'CA-AB', 'CA-MB', 'CA-BC', 'US-VT',
       'US-OR', 'US-AR', 'US-AK', 'US-HI', 'US-MT', 'US-UT', 'CA-NL',
       'CA-QC', 'US-WY', 'US-MS', 'US-LA', 'CA-SK', 'US-RI', 'CA-YT',
       'CA-NB', 'CA-NS', 'US-DC', 'US-NV', 'US-ND', 'CA-PE', 'CA-NT'],
      dtype=object)

In [5]:
preview.Year.unique()

array([1992, 1991, 1989, 1988, 1994, 1995, 1993, 1990, 1987], dtype=int64)

---
## Trimming it down to the essentials
There is an extraordinary amount of information, but it's also much more than is needed for this project. At this stage, and with the help of an enirely different dataset, also provided by FeederWatch, everything else can be derived from only four columns.

In [6]:
# specify necessary columns
preview_trimmed_columns = ['SUBNATIONAL1_CODE', 'Year', 'SPECIES_CODE', 'HOW_MANY']

#load trimmed dataset
preview_trimmed = pd.read_csv('bird_datasets/Project FeederWatch/all_years/PFW_all_1988_1995_June2023_Public.csv',
                                usecols = preview_trimmed_columns)
preview_trimmed.head()

,SUBNATIONAL1_CODE,Year,SPECIES_CODE,HOW_MANY
0,US-MI,1992,houfin,4.0
1,US-MI,1992,houspa,26.0
2,US-MI,1992,moudov,5.0
3,US-MI,1992,eursta,35.0
4,US-MI,1992,daejun,2.0


---
## Creating the row filter
Ultimately, seven large datasets will be concatenated into one. Even with keeping only four columns, this would still result in a massive file, much of which wouldn't be needed for this study.

The focus of this project is cavity nesting birds, so those that don't fall under that classification can be filtered out. Additionally, cavity nesters which wouldn't be reasonably expected to be affected by the invasive cavity nesters can be eliminated from the data. Species which include waterfowl, owls, and even peregrine falcons, among others, can be left aside from this study. *(A European starling facing off against a peregrine falcon, if it were to happen, would be a very short confrontation.)*

FeederWatch has provided a table to translate the **SPECIES_CODE** column. This will be used in creating the row filter for all the datasets, as well as in an upcoming step.

In [7]:
#load species code translator dataset

species_codes = pd.read_csv('PFW_spp_translation_table_May2023.csv')
species_codes.head()

,species_code,alt_full_spp_code,n_locations,scientific_name,american_english_name,taxonomy_version,taxonomic_sort_order
0,grerhe1,NaN,1,Rhea americana,Greater Rhea,2022,10
1,gretin1,NaN,3,Tinamus major,Great Tinamou,2022,42
2,wfwduc1,NaN,1,Dendrocygna viduata,White-faced Whistling-Duck,2022,228
3,bbwduc,NaN,105,Dendrocygna autumnalis,Black-bellied Whistling-Duck,2022,230
4,fuwduc,NaN,2,Dendrocygna bicolor,Fulvous Whistling-Duck,2022,235


### Only the species code and common names will be needed from this dataset, so a subset of it will suffice.

In [8]:
# create species_codes subset

species_codes_sub = species_codes[['species_code', 'american_english_name']]

species_codes_sub.head()

,species_code,american_english_name
0,grerhe1,Greater Rhea
1,gretin1,Great Tinamou
2,wfwduc1,White-faced Whistling-Duck
3,bbwduc,Black-bellied Whistling-Duck
4,fuwduc,Fulvous Whistling-Duck


In [9]:
species_codes_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   species_code           1090 non-null   object
 1   american_english_name  1090 non-null   object
dtypes: object(2)
memory usage: 17.2+ KB


---
Again, only the species relevant to the project will be needed. Armed with prior knowledge, Google, and Wikipedia, I was able to compile a list of the cavity nesting species from the FeederWatch data that fit the criteria of the project. I then turned it into a .csv file. This list includes only the common names and matches with the **american_english_name** column of the translator dataset.

In [10]:
# load list of relevant species

species_filter = pd.read_csv('c_nester_list.csv')

species_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    69 non-null     object
dtypes: object(1)
memory usage: 684.0+ bytes


In [11]:
species_filter_list = species_filter.name.tolist()
species_filter_list[:9]

['Arizona Woodpecker',
 'Ash-throated Flycatcher',
 "Bewick's Wren",
 "Bewick's Wren (bewickii)",
 "Bewick's Wren (spilurus Group)",
 'Black-backed Woodpecker',
 'Black-capped Chickadee',
 'Boreal Chickadee',
 'Bridled Titmouse']

### Now to filter the translator dataset.

In [12]:
# filter species_codes_sub rows to inlcude only rows from filter_list

species_codes_sub_filtered = species_codes_sub.loc[species_codes_sub['american_english_name'].isin(species_filter_list)].sort_values(by = ['american_english_name'], ascending = True)

species_codes_sub_filtered = species_codes_sub_filtered.reset_index(drop = True)

species_codes_sub_filtered.head()

,species_code,american_english_name
0,ariwoo,Arizona Woodpecker
1,astfly,Ash-throated Flycatcher
2,bewwre,Bewick's Wren
3,bewwre3,Bewick's Wren (bewickii)
4,bewwre1,Bewick's Wren (spilurus Group)


In [13]:
species_codes_sub_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   species_code           69 non-null     object
 1   american_english_name  69 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [14]:
# check length of dataset against filter list

len(species_filter_list) - species_codes_sub['american_english_name'].isin(species_filter_list).sum()

0

### Now that the filter for the filter filter has been filtered and the filtered filter filter has been filtered, the next step is to create the final row filter.

In [42]:
#create final row filter

row_filter = species_codes_sub_filtered.species_code.to_list()

row_filter[:5]

['ariwoo', 'astfly', 'bewwre', 'bewwre3', 'bewwre1']

---
## Creating the initial working dataset
Now that the columns from the raw datasets have been chosen and the row filter has been created, the task of building a single initial working dataset can be tackled.

In [51]:
# Concatenate all datasets with filtered columns and rows

files = glob.glob('bird_datasets/Project FeederWatch/all_years/PFW_all_*.csv', recursive = True)


df_list = []
for filename in files:
  file = pd.read_csv(filename, usecols = preview_trimmed_columns)
  data = file.loc[file['SPECIES_CODE'].isin(row_filter)]
  df_list.append(data)

  

birds_main_initial = pd.concat(df_list)
birds_main_initial.reset_index(drop = True, inplace = True)
birds_main_initial.head()

,SUBNATIONAL1_CODE,Year,SPECIES_CODE,HOW_MANY
0,US-MI,1992,houfin,4.0
1,US-MI,1992,houspa,26.0
2,US-MI,1992,eursta,35.0
3,US-MI,1992,bkcchi,1.0
4,US-MI,1992,dowwoo,1.0


---
### Inspecting the new dataframe

In [52]:
birds_main_initial.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19832302 entries, 0 to 19832301
Data columns (total 4 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   SUBNATIONAL1_CODE  19832302 non-null  object 
 1   Year               19832302 non-null  int64  
 2   SPECIES_CODE       19832302 non-null  object 
 3   HOW_MANY           19832302 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 605.2+ MB


In [54]:
# check row filter functionality

birds_main_initial['SPECIES_CODE'].nunique() - len(row_filter)

0

In [76]:
#check included years

years = birds_main_initial['Year'].unique()
years_sorted = years.sort()
years

array([1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022, 2023, 2024], dtype=int64)

In [77]:
# rename Year for consistency

birds_main_initial.columns = ['SUBNATIONAL1_CODE', 'YEAR', 'SPECIES_CODE', 'HOW_MANY']

birds_main_initial.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19832302 entries, 0 to 19832301
Data columns (total 4 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   SUBNATIONAL1_CODE  19832302 non-null  object 
 1   YEAR               19832302 non-null  int64  
 2   SPECIES_CODE       19832302 non-null  object 
 3   HOW_MANY           19832302 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 605.2+ MB


In [79]:
# check for missing data

total = birds_main_initial.isnull().sum().sort_values(ascending=False)
percent = (birds_main_initial.isnull().sum()/birds_main_initial.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total Missing', 'Percent'])
missing_data

,Total Missing,Percent
SUBNATIONAL1_CODE,0,0.0
YEAR,0,0.0
SPECIES_CODE,0,0.0
HOW_MANY,0,0.0


---
### Creating the COMMON_NAME column
Most people aren't familiar with the 6-character [BBL alpha codes](https://www.birdpop.org/pages/birdSpeciesCodes.php) that represent the bird species in the existing table. A column for what we non-ornithologists call these birds would be most helpful.

### Creating a decoder
Before this new column can be created, there has to be a way to "decode" the **SPECIES_CODE** column. This is where the custom **species_codes_sub_filtered** dataset comes into play for its second task. This time it's used to create a dictionary to serve as a decoder.

In [80]:
# create species name decoder dictionary

decoder = dict(zip(species_codes_sub_filtered['species_code'],species_codes_sub_filtered['american_english_name']))

list(decoder.items())[:10]

[('ariwoo', 'Arizona Woodpecker'),
 ('astfly', 'Ash-throated Flycatcher'),
 ('bewwre', "Bewick's Wren"),
 ('bewwre3', "Bewick's Wren (bewickii)"),
 ('bewwre1', "Bewick's Wren (spilurus Group)"),
 ('bkbwoo', 'Black-backed Woodpecker'),
 ('bkcchi', 'Black-capped Chickadee'),
 ('borchi2', 'Boreal Chickadee'),
 ('britit', 'Bridled Titmouse'),
 ('brncre', 'Brown Creeper')]

### Now to create and populate the COMMON_NAME column

In [81]:
# populate new column

new_col = []

for item in birds_main_initial['SPECIES_CODE']:
    for key in decoder.keys():
        if item == key:
            new_col.append(decoder[item])
       
            
birds_main_initial['COMMON_NAME'] = new_col

birds_main_initial.head()

,SUBNATIONAL1_CODE,YEAR,SPECIES_CODE,HOW_MANY,COMMON_NAME
0,US-MI,1992,houfin,4.0,House Finch
1,US-MI,1992,houspa,26.0,House Sparrow
2,US-MI,1992,eursta,35.0,European Starling
3,US-MI,1992,bkcchi,1.0,Black-capped Chickadee
4,US-MI,1992,dowwoo,1.0,Downy Woodpecker


### Keeping it all together
Moving **SPECIES_CODE** and **COMMON_NAME** side-by-side, for easier reading and improved comprehension.

In [82]:
# reorder columns

birds_main_initial_v2 = birds_main_initial[['SUBNATIONAL1_CODE', 'YEAR',
       'SPECIES_CODE', 'COMMON_NAME', 'HOW_MANY']]

birds_main_initial_v2.head()

,SUBNATIONAL1_CODE,YEAR,SPECIES_CODE,COMMON_NAME,HOW_MANY
0,US-MI,1992,houfin,House Finch,4.0
1,US-MI,1992,houspa,House Sparrow,26.0
2,US-MI,1992,eursta,European Starling,35.0
3,US-MI,1992,bkcchi,Black-capped Chickadee,1.0
4,US-MI,1992,dowwoo,Downy Woodpecker,1.0


In [83]:
birds_main_initial_v2.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19832302 entries, 0 to 19832301
Data columns (total 5 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   SUBNATIONAL1_CODE  19832302 non-null  object 
 1   YEAR               19832302 non-null  int64  
 2   SPECIES_CODE       19832302 non-null  object 
 3   COMMON_NAME        19832302 non-null  object 
 4   HOW_MANY           19832302 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 756.5+ MB


---
## Creating an easier-to-handle file for further work
To avoid having to run those cells that have done so much heavy lifting already, creating a smaller working "checkpoint" file will provide a more efficient starting point for further work on this dataset. It also preserves the current dataset for future projects using similar data.

In [25]:
# create a checkpoint .csv file

birds_main_initial_v2.to_csv('inv_birds_checkpoint_1.csv', index = None, header=True)

---
## A fresh start
Having a smaller file to work with which contains only the essentials with will improve efficiency.

### Load checkpoint_1.csv

In [119]:
# load .csv from checkpoint

birds_main_secondary = pd.read_csv('inv_birds_checkpoint_1.csv')
birds_main_secondary.head()

,SUBNATIONAL1_CODE,YEAR,SPECIES_CODE,COMMON_NAME,HOW_MANY
0,US-MI,1992,houfin,House Finch,4.0
1,US-MI,1992,houspa,House Sparrow,26.0
2,US-MI,1992,eursta,European Starling,35.0
3,US-MI,1992,bkcchi,Black-capped Chickadee,1.0
4,US-MI,1992,dowwoo,Downy Woodpecker,1.0


In [120]:
birds_main_secondary.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19832302 entries, 0 to 19832301
Data columns (total 5 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   SUBNATIONAL1_CODE  19832302 non-null  object 
 1   YEAR               19832302 non-null  int64  
 2   SPECIES_CODE       19832302 non-null  object 
 3   COMMON_NAME        19832302 non-null  object 
 4   HOW_MANY           19832302 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 756.5+ MB


### Further reduction in file size is needed
In its current state, this file contains nearly 20 million observations and has a file size of over 756 MB. GitHub won't be having that. This would be a good time to aggregate values of the **HOW_MANY** column.

In [121]:
# add totals and averages
birds_main_sec_2 = birds_main_secondary.groupby(['SUBNATIONAL1_CODE', 'YEAR', 'SPECIES_CODE', 'COMMON_NAME']).agg(
    TOTAL_COUNTED = ('HOW_MANY', 'sum'), AVERAGE_COUNTED = ('HOW_MANY', 'mean')).reset_index()

birds_main_sec_2.head()

,SUBNATIONAL1_CODE,YEAR,SPECIES_CODE,COMMON_NAME,TOTAL_COUNTED,AVERAGE_COUNTED
0,CA-AB,1988,bkcchi,Black-capped Chickadee,301.0,6.688889
1,CA-AB,1988,borchi2,Boreal Chickadee,11.0,2.200000
2,CA-AB,1988,brncre,Brown Creeper,1.0,1.000000
3,CA-AB,1988,chbchi,Chestnut-backed Chickadee,22.0,22.000000
4,CA-AB,1988,dowwoo,Downy Woodpecker,39.0,1.300000


---
### Splitting SUBNATIONAL1_CODE into separate columns: COUNTRY and STATE/PROVINCE
For consideration of both analysis and eye appeal, this geographical data would be better presented as separate columns.

### A preliminary check for invalid codes

In [122]:
birds_main_sec_2['SUBNATIONAL1_CODE'].unique()

array(['CA-AB', 'CA-BC', 'CA-MB', 'CA-NB', 'CA-NL', 'CA-NS', 'CA-NT',
       'CA-ON', 'CA-PE', 'CA-QC', 'CA-SK', 'CA-YT', 'MN-061', 'MX-JAL',
       'MX-NAY', 'PM-', 'US-AK', 'US-AL', 'US-AR', 'US-AZ', 'US-CA',
       'US-CO', 'US-CT', 'US-DC', 'US-DE', 'US-FL', 'US-GA', 'US-HI',
       'US-IA', 'US-ID', 'US-IL', 'US-IN', 'US-KS', 'US-KY', 'US-LA',
       'US-MA', 'US-MD', 'US-ME', 'US-MI', 'US-MN', 'US-MO', 'US-MS',
       'US-MT', 'US-NC', 'US-ND', 'US-NE', 'US-NH', 'US-NJ', 'US-NM',
       'US-NV', 'US-NY', 'US-OH', 'US-OK', 'US-OR', 'US-PA', 'US-RI',
       'US-SC', 'US-SD', 'US-TN', 'US-TX', 'US-UT', 'US-VA', 'US-VT',
       'US-WA', 'US-WI', 'US-WV', 'US-WY', 'XX-'], dtype=object)

##### 'MN-061' and 'XX-' are invalid. 'PM-' is Saint Pierre and Miquelon.

#### First to fix 'PM-', and then remove rows with invalid codes.

In [123]:
birds_main_sec_2['SUBNATIONAL1_CODE'] = birds_main_sec_2['SUBNATIONAL1_CODE'].replace('PM-', 'PM-PM', regex=True)

In [124]:
birds_main_sec_2 = birds_main_sec_2.drop(birds_main_sec_2[birds_main_sec_2['SUBNATIONAL1_CODE'] == 'MN-061'].index)
birds_main_sec_2 = birds_main_sec_2.drop(birds_main_sec_2[birds_main_sec_2['SUBNATIONAL1_CODE'] == 'XX-'].index)

birds_main_sec_2['SUBNATIONAL1_CODE'].unique()

array(['CA-AB', 'CA-BC', 'CA-MB', 'CA-NB', 'CA-NL', 'CA-NS', 'CA-NT',
       'CA-ON', 'CA-PE', 'CA-QC', 'CA-SK', 'CA-YT', 'MX-JAL', 'MX-NAY',
       'PM-PM', 'US-AK', 'US-AL', 'US-AR', 'US-AZ', 'US-CA', 'US-CO',
       'US-CT', 'US-DC', 'US-DE', 'US-FL', 'US-GA', 'US-HI', 'US-IA',
       'US-ID', 'US-IL', 'US-IN', 'US-KS', 'US-KY', 'US-LA', 'US-MA',
       'US-MD', 'US-ME', 'US-MI', 'US-MN', 'US-MO', 'US-MS', 'US-MT',
       'US-NC', 'US-ND', 'US-NE', 'US-NH', 'US-NJ', 'US-NM', 'US-NV',
       'US-NY', 'US-OH', 'US-OK', 'US-OR', 'US-PA', 'US-RI', 'US-SC',
       'US-SD', 'US-TN', 'US-TX', 'US-UT', 'US-VA', 'US-VT', 'US-WA',
       'US-WI', 'US-WV', 'US-WY'], dtype=object)

### Performing the split

In [125]:
split = birds_main_sec_2['SUBNATIONAL1_CODE'].str.split('-')

birds_main_sec_2['COUNTRY'] = split.str.get(0)
birds_main_sec_2['STATE/PROVINCE'] = split.str.get(1)

birds_main_sec_2.head()

,SUBNATIONAL1_CODE,YEAR,SPECIES_CODE,COMMON_NAME,TOTAL_COUNTED,AVERAGE_COUNTED,COUNTRY,STATE/PROVINCE
0,CA-AB,1988,bkcchi,Black-capped Chickadee,301.0,6.688889,CA,AB
1,CA-AB,1988,borchi2,Boreal Chickadee,11.0,2.200000,CA,AB
2,CA-AB,1988,brncre,Brown Creeper,1.0,1.000000,CA,AB
3,CA-AB,1988,chbchi,Chestnut-backed Chickadee,22.0,22.000000,CA,AB
4,CA-AB,1988,dowwoo,Downy Woodpecker,39.0,1.300000,CA,AB


### Checking the values

In [126]:
birds_main_sec_2['COUNTRY'].unique()

array(['CA', 'MX', 'PM', 'US'], dtype=object)

In [127]:
states_list = birds_main_sec_2['STATE/PROVINCE'].unique()
states_list.sort()
states_list

array(['AB', 'AK', 'AL', 'AR', 'AZ', 'BC', 'CA', 'CO', 'CT', 'DC', 'DE',
       'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'JAL', 'KS', 'KY', 'LA',
       'MA', 'MB', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NAY', 'NB',
       'NC', 'ND', 'NE', 'NH', 'NJ', 'NL', 'NM', 'NS', 'NT', 'NV', 'NY',
       'OH', 'OK', 'ON', 'OR', 'PA', 'PE', 'PM', 'QC', 'RI', 'SC', 'SD',
       'SK', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'YT'],
      dtype=object)

---
### Adding the  binary column INVASIVE
As the focus of this project is *invasive* cavity nesting birds, a variable to indicate that characteristic will be useful.  
**1 = invasive, 0 = non-invasive**

In [128]:
invasive_code = ['houspa', 'eursta', 'eutspa']

birds_main_sec_2['INVASIVE'] = birds_main_sec_2['SPECIES_CODE'].isin(invasive_code).map({True: 1, False: 0})

birds_main_sec_2.head()

,SUBNATIONAL1_CODE,YEAR,SPECIES_CODE,COMMON_NAME,TOTAL_COUNTED,AVERAGE_COUNTED,COUNTRY,STATE/PROVINCE,INVASIVE
0,CA-AB,1988,bkcchi,Black-capped Chickadee,301.0,6.688889,CA,AB,0
1,CA-AB,1988,borchi2,Boreal Chickadee,11.0,2.200000,CA,AB,0
2,CA-AB,1988,brncre,Brown Creeper,1.0,1.000000,CA,AB,0
3,CA-AB,1988,chbchi,Chestnut-backed Chickadee,22.0,22.000000,CA,AB,0
4,CA-AB,1988,dowwoo,Downy Woodpecker,39.0,1.300000,CA,AB,0


## Final assembly
With all the foreseeable necessary variables created and the observations filtered and aggregated, it's time for a some **5S** methodology to bring it all together.

In [129]:
# remove 'SUBNATIONAL1_CODE' and reorder columns

birds_main = birds_main_sec_2[['YEAR', 'COUNTRY', 'STATE/PROVINCE', 'SPECIES_CODE', 'COMMON_NAME',
        'INVASIVE', 'TOTAL_COUNTED', 'AVERAGE_COUNTED']].sort_values(by = ['YEAR'], ascending = True)
birds_main.reset_index(drop = True, inplace = True)

birds_main.head()

,YEAR,COUNTRY,STATE/PROVINCE,SPECIES_CODE,COMMON_NAME,INVASIVE,TOTAL_COUNTED,AVERAGE_COUNTED
0,1987,US,NJ,tuftit,Tufted Titmouse,0,6.0,2.00
1,1987,US,NJ,houspa,House Sparrow,1,195.0,39.00
2,1987,US,NJ,houfin,House Finch,0,38.0,7.60
3,1987,US,NJ,eursta,European Starling,1,61.0,12.20
4,1987,US,NJ,dowwoo,Downy Woodpecker,0,7.0,1.75


In [130]:
birds_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41481 entries, 0 to 41480
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             41481 non-null  int64  
 1   COUNTRY          41481 non-null  object 
 2   STATE/PROVINCE   41481 non-null  object 
 3   SPECIES_CODE     41481 non-null  object 
 4   COMMON_NAME      41481 non-null  object 
 5   INVASIVE         41481 non-null  int64  
 6   TOTAL_COUNTED    41481 non-null  float64
 7   AVERAGE_COUNTED  41481 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 2.5+ MB


### Another check for missing data

In [131]:
total = birds_main.isnull().sum().sort_values(ascending=False)
percent = (birds_main.isnull().sum()/birds_main.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total Missing', 'Percent'])

missing_data

,Total Missing,Percent
YEAR,0,0.0
COUNTRY,0,0.0
STATE/PROVINCE,0,0.0
SPECIES_CODE,0,0.0
COMMON_NAME,0,0.0
INVASIVE,0,0.0
TOTAL_COUNTED,0,0.0
AVERAGE_COUNTED,0,0.0


---
## Writing the final draft to .csv
With the final assembly complete and the file size now at a mere and very GitHub-friendly 2.5+ MB, it's ready to fly. (Pun intended.) 

In [132]:
birds_main.to_csv('invasive_birds_final.csv', index = None, header=True)

---
## Loading and inspecting the new working file

In [133]:
birds_df = pd.read_csv('invasive_birds_final.csv')

birds_df.head()

,YEAR,COUNTRY,STATE/PROVINCE,SPECIES_CODE,COMMON_NAME,INVASIVE,TOTAL_COUNTED,AVERAGE_COUNTED
0,1987,US,NJ,tuftit,Tufted Titmouse,0,6.0,2.00
1,1987,US,NJ,houspa,House Sparrow,1,195.0,39.00
2,1987,US,NJ,houfin,House Finch,0,38.0,7.60
3,1987,US,NJ,eursta,European Starling,1,61.0,12.20
4,1987,US,NJ,dowwoo,Downy Woodpecker,0,7.0,1.75


In [134]:
birds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41481 entries, 0 to 41480
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             41481 non-null  int64  
 1   COUNTRY          41481 non-null  object 
 2   STATE/PROVINCE   41481 non-null  object 
 3   SPECIES_CODE     41481 non-null  object 
 4   COMMON_NAME      41481 non-null  object 
 5   INVASIVE         41481 non-null  int64  
 6   TOTAL_COUNTED    41481 non-null  float64
 7   AVERAGE_COUNTED  41481 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 2.5+ MB


In [135]:
birds_df.describe()

,YEAR,INVASIVE,TOTAL_COUNTED,AVERAGE_COUNTED
count,41481.000000,41481.000000,4.148100e+04,41481.000000
mean,2006.573781,0.111666,8.193680e+03,4.018409
std,10.593436,0.314958,1.309572e+06,127.369894
min,1987.000000,0.000000,1.000000e+00,1.000000
25%,1998.000000,0.000000,1.500000e+01,1.196273
50%,2007.000000,0.000000,1.490000e+02,1.642692
75%,2016.000000,0.000000,1.126000e+03,3.257143
max,2024.000000,1.000000,2.667188e+08,25804.838719


In [136]:
# check for missing data

total = birds_df.isnull().sum().sort_values(ascending=False)
percent = (birds_df.isnull().sum()/birds_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total Missing', 'Percent'])
missing_data

,Total Missing,Percent
YEAR,0,0.0
COUNTRY,0,0.0
STATE/PROVINCE,0,0.0
SPECIES_CODE,0,0.0
COMMON_NAME,0,0.0
INVASIVE,0,0.0
TOTAL_COUNTED,0,0.0
AVERAGE_COUNTED,0,0.0
